## Homework: Vector Search

In this homework, I'll experiemnt with vector with and without Elasticsearch

## Q1. Getting the embeddings model

First, I will get the embeddings model `multi-qa-distilbert-cos-v1` from
[the Sentence Transformer library](https://www.sbert.net/docs/sentence_transformer/pretrained_models.html#model-overview)


In [1]:
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer("multi-qa-distilbert-cos-v1")

c:\Users\marti.MARTIN\AppData\Local\Programs\Python\Python312\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
c:\Users\marti.MARTIN\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\marti.MARTIN\.cache\huggingface\hub\models--sentence-transformers--multi-qa-distilbert-cos-v1. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlink

Create the embedding for this user question:

In [4]:
user_question = "I just discovered the course. Can I still join it?"
embedding_model.encode(user_question)[0]

0.07822262

What's the first value of the resulting vector?

* -0.24
* -0.04
* **0.07**
* 0.27

## Prepare the documents

Now I will create the embeddings for the documents.

I load the documents with ids that have been prepared in the module:

In [5]:
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

We will use only a subset of the questions - the questions
for `"machine-learning-zoomcamp"`. After filtering, you should
have only 375 documents

In [8]:
documents[1]

{'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
 'section': 'General course-related questions',
 'question': 'Course - What are the prerequisites for this course?',
 'course': 'data-engineering-zoomcamp',
 'id': '1f6520ca'}

In [11]:
subset_docs = [item for item in documents if item['course'] == 'machine-learning-zoomcamp']
len(subset_docs)

375

## Q2. Creating the embeddings

Now for each document, we will create an embedding for both question and answer fields.

We want to put all of them into a single matrix `X`:

- Create a list `embeddings` 
- Iterate over each document 
- `qa_text = f'{question} {text}'`
- compute the embedding for `qa_text`, append to `embeddings`
- At the end, let `X = np.array(embeddings)` (`import numpy as np`) 

What's the shape of X? (`X.shape`). Include the parantheses. 

In [12]:
import numpy as np

In [28]:
embeddings = []
for doc in subset_docs:
    qa_text = f"{doc['question']} {doc['text']}"
    embedding = embedding_model.encode(qa_text)
    embeddings.append(embedding)

X = np.array(embeddings)

In [29]:
print(X)
print(X.shape)

[[ 0.08805911  0.01559365  0.07925579 ...  0.00745124  0.00241913
   0.01147132]
 [ 0.07066914 -0.01930562  0.0773837  ...  0.04495562  0.01078554
  -0.02316591]
 [ 0.10229155 -0.0166324   0.03414484 ...  0.03215319 -0.04317445
  -0.04574377]
 ...
 [-0.00768201  0.01075665  0.01190489 ...  0.06066166 -0.03393208
   0.01605328]
 [ 0.13408284 -0.03755201  0.0197653  ...  0.02163629 -0.01921146
   0.03690196]
 [ 0.04386011 -0.0130077   0.06373127 ...  0.03338454  0.02258543
  -0.0630587 ]]
(375, 768)


## Q3. Search

We have the embeddings and the query vector. Now let's compute the 
cosine similarity between the vector from Q1 (let's call it `v`) and the matrix from Q2. 

The vectors returned from the embedding model are already
normalized (you can check it by computing a dot product of a vector
with itself - it should return 1.0). This means that in order
to compute the coside similarity, it's sufficient to 
multiply the matrix `X` by the vector `v`:


In [31]:
v = embedding_model.encode(user_question)

In [33]:
v.dot(v)

0.9999999

In [40]:
scores = X.dot(v)
scores.max()

0.65065753

What's the highest score in the results?

- 65.0 
- 6.5
- **0.65**
- 0.065